In [6]:
import pandas as pd
import numpy as np
import zipfile
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

import akshare as ak

## utility

# 获取沪深300成分股
hs300_stock_list = ak.index_stock_cons(symbol="000300")
stock_list = hs300_stock_list['品种代码']
hs300_list = list(stock_list)

## 获取所有股票上市时间
def get_sh_time():    
    df_sh = pd.read_excel('../量化data/股票代码.xlsx')
    df_sh = df_sh.iloc[0:5355,:]
    df_sh = df_sh.rename(columns={'证券代码↑':'code'})
    df_sh['上市日期'] = pd.to_datetime(df_sh['上市日期']) 
    return df_sh

# 1 MAD:中位数去极值
def extreme_MAD(dt,n = 3):
    """
    args:
        dt:一列或多列时间序列,df
        n : 中位数+-n*偏差中位数
    return:
        返回替换极值dt
    """
    median = dt.quantile(0.5)   # 找出中位数
    new_median = (abs((dt - median)).quantile(0.5))   # 偏差值的中位数
    dt_up = median + n*new_median    # 上限
    dt_down = median - n*new_median  # 下限
    return dt.clip(dt_down, dt_up, axis=1)    # 超出上下限的值，赋值为上下限

# 2 填补缺失值
def Na_Fill(df,method='ffill'):
    """
    args:
        df:df格式数据
        method: 填补方式，'ffill'前一个非NA值填充,'bfill'后一个非NA填充
    return:
        df
    """
    df = df.fillna(method)
    return df

def get_sh_process(x,df_sh):
    """
    去除上市前的样本
    args:
        x: 单只股票的df
        df_sh:记录股票上市时间的df
    return:
        单只股票去除上市前样本的df
    """
    t = df_sh[df_sh['code']==x.iloc[0,1]]['上市日期'].values
    if t.size != 0:
        condition = x['date'].values > t
        x = x[condition]
    else:
        x = x.loc[x['上市状态_Listedstate'].isin(['Norm', '*ST'])]
    return x


def Stock_Pool_split(df):
    """
    股票池筛选：
    (1) 剔除上市不足1年
    (2) 剔除ST
    (3) 剔除每股净资产为负
    """

    ## 根据'code'列分组，并将每个分组的DataFrame添加到列表中
    grouped_dfs = [group_df for _, group_df in df.groupby('code')]
    df_sh = get_sh_time()
    y = [] #记录每支股票的信息
    l = [] #记录每只股票样本长度
    ST = [] # 记录是否被ST过
    neg_res = [] # 记录净资产是否为负
    hs300 = [] # 记录是否沪深300的股票

    for x in grouped_dfs:

        ## 去除上市前的样本
        x = get_sh_process(x,df_sh)
        ## 记录样本长度
        l.append(x.shape[0])
        ## 记录是否被ST
        con_st = x['上市状态_Listedstate'].str.contains('ST').any()
        ST.append(con_st)
        ## 记录净资产是否为负过
        num = (x['每股净资产(元/股)_NAPS']<0).sum()
        neg_res.append(num>0)
        # ## 记录是否沪深300的股票
        if x.shape[0] == 0:
            con_hs300 = False
        else:
            con_hs300 = x['code'].values[0][:6] in hs300_list
        hs300.append(con_hs300)


        y.append(x)
    
    ## 剔除上市小于1年的、被ST的、净资产负过的股票
    condition = [con_1 and not con_2 and not con_3 and con_4 for con_1, con_2,con_3, con_4 in zip(list(np.array(l) >=365), ST, neg_res,hs300)]
    grouped_sel = [item for item, con in zip(y,condition) if con]

    return  pd.concat(grouped_sel, ignore_index=True)


def Standard_process(df):
    """
    对输入的所有股票的df进行如下操作:
    (1) 对每只股票进行：极端值、缺失值处理
    (2) 对每只股票的数据进行标准化
    """
    
    ## 根据'code'列分组，并将每个分组的DataFrame添加到列表中
    grouped_sel= [group_df for _, group_df in df.groupby('code')]

    y=[]
    for x in grouped_sel:
        ## 替换极端值
        x.iloc[:,4:x.shape[1]] = extreme_MAD(x.iloc[:,4:x.shape[1]],n = 3)
        ## 填补缺失值
        x = x.apply(lambda x: x.fillna(method='ffill'))
        ## z_score 标准化
        scaler = StandardScaler()
        x.iloc[:,4:] = scaler.fit_transform(x.iloc[:,4:])

        y.append(x)

    return pd.concat(y, ignore_index=True)

def get_y(df):
    
    pass


In [7]:
df = pd.read_csv('../量化data/股票整合数据_未预处理.zip', compression='zip')
df['date'] = pd.to_datetime(df['date'])
df_1 = Stock_Pool_split(df)
# grouped_dfs = [group_df for _, group_df in df_1.groupby('code')]

In [10]:
columns = ['date', 'code','Open', 'High', 'Low', 'Close','累积股价调整乘子_Mcfacpr','日收益率_Dret',
           '成交量_Trdvol','成交金额_Trdsum','已上市流通股_Lsttrdshr']
df_2 = df_1[columns]
df_2 = df_2.rename(columns={'code':'stock_code','Open':'open', 'High':'high', 'Low':'low', 'Close':'close',
                            '累积股价调整乘子_Mcfacpr':'factor','日收益率_Dret':'change','成交量_Trdvol':'volume',
                            '成交金额_Trdsum':'money','已上市流通股_Lsttrdshr':'流通股'})
df_2 


,date,stock_code,open,high,low,close,factor,change,volume,money,流通股
0,2021-02-01,000001.SZ,21.8838,23.7772,21.5984,23.3586,122.9998,0.0632,147523930.0,3.529557e+09,1.940576e+10
1,2021-02-02,000001.SZ,22.2454,22.9304,21.7982,22.1502,122.9998,-0.0517,241616877.0,5.679180e+09,1.940576e+10
2,2021-02-03,000001.SZ,22.3405,23.9580,22.2454,23.7392,122.9998,0.0717,192327159.0,4.690176e+09,1.940576e+10
3,2021-02-04,000001.SZ,23.0066,24.0151,22.8734,23.4062,122.9998,-0.0140,125524750.0,3.084554e+09,1.940576e+10
4,2021-02-05,000001.SZ,23.4062,24.0817,23.0922,23.7202,122.9998,0.0134,101557559.0,2.517804e+09,1.940576e+10
...,...,...,...,...,...,...,...,...,...,...,...
183242,2023-12-25,605499.SH,176.9300,177.3000,172.6900,175.6200,1.0318,-0.0044,737300.0,1.290807e+08,1.592709e+08
183243,2023-12-26,605499.SH,174.8700,176.9500,174.3400,175.6300,1.0318,0.0001,453300.0,7.969934e+07,1.592709e+08
183244,2023-12-27,605499.SH,175.6800,178.4700,175.6700,177.5500,1.0318,0.0109,671856.0,1.190336e+08,1.592709e+08
183245,2023-12-28,605499.SH,177.9500,178.4700,174.8000,176.9600,1.0318,-0.0033,808121.0,1.425958e+08,1.592709e+08


In [ ]:
df_2.to_csv('../量化data/df_hs.csv')

In [29]:
# ## 输出每只股票的csv
# codes = df_2['stock_code'].unique()
# for i in codes:
#     df = df_2.loc[df_2['stock_code'] == i]
    
#     #将每支股票的相关信息分别导出成一个csv文件
#     df.to_csv('C:/Users/Lincoln/.qlib/csv_data/ch_data/'+ i +'.csv',header = True,encoding='utf_8_sig',index=False)

In [ ]:
# python scripts/dump_bin.py dump_all --csv_path ~/.qlib/csv_data/ch_data --qlib_dir ~/.qlib/qlib_data/ch_data --symbol_field_name stock_code --date_field_name date --include_fields name,volume,money,factor,close,open,high,low,change